<a href="https://colab.research.google.com/github/GOURAVRISHI/NLP_lectures/blob/main/text_generation_21may.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment
Use DVC or MLFlow to convert this into 
1. Create and API which will take the first input and it'll generate the next sentence 
or
2. Create and UI for this app

In [1]:
import os
ROOT= "/content/drive/MyDrive/Colab Notebooks/text_generation"
os.chdir(ROOT)

# Text Generation with RNN

In [2]:
import tensorflow as tf
import numpy as np

import time

In [3]:
os.makedirs("data", exist_ok= True)

In [4]:
!curl https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt > "./data/shakespeare.txt"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1089k  100 1089k    0     0  1525k      0 --:--:-- --:--:-- --:--:-- 1525k


In [5]:
# same thing can be done using Config.yaml, also use constructor, 
class Config:
  path_to_file = os.path.join("data", "shakespeare.txt")
  seq_length = 100

  Batch_size= 64
  Buffer_size = 10000 # keep things ready for training in buffer

  embedding_dim= 256

  rnn_units= 1024

  EPOCHS = 30

  checkpoint_dir= "./training_ckpt"

In [6]:
Config.EPOCHS

30

In [7]:
# Reading first few lines of the text
text = open(Config.path_to_file, "rb").read().decode(encoding= "utf-8")
text[:100]

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'

In [8]:
with open(Config.path_to_file) as f:
  t= f.read()

In [9]:
t[:100]

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'

In [10]:
set(t)

{'\n',
 ' ',
 '!',
 '$',
 '&',
 "'",
 ',',
 '-',
 '.',
 '3',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z'}

In [11]:
sorted(set(t))

['\n',
 ' ',
 '!',
 '$',
 '&',
 "'",
 ',',
 '-',
 '.',
 '3',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [12]:
vocab = sorted(set(t))
len(vocab)

65

In [13]:
char2idx ={uniChar: idx for idx, uniChar in enumerate(vocab)}
char2idx

{'\n': 0,
 ' ': 1,
 '!': 2,
 '$': 3,
 '&': 4,
 "'": 5,
 ',': 6,
 '-': 7,
 '.': 8,
 '3': 9,
 ':': 10,
 ';': 11,
 '?': 12,
 'A': 13,
 'B': 14,
 'C': 15,
 'D': 16,
 'E': 17,
 'F': 18,
 'G': 19,
 'H': 20,
 'I': 21,
 'J': 22,
 'K': 23,
 'L': 24,
 'M': 25,
 'N': 26,
 'O': 27,
 'P': 28,
 'Q': 29,
 'R': 30,
 'S': 31,
 'T': 32,
 'U': 33,
 'V': 34,
 'W': 35,
 'X': 36,
 'Y': 37,
 'Z': 38,
 'a': 39,
 'b': 40,
 'c': 41,
 'd': 42,
 'e': 43,
 'f': 44,
 'g': 45,
 'h': 46,
 'i': 47,
 'j': 48,
 'k': 49,
 'l': 50,
 'm': 51,
 'n': 52,
 'o': 53,
 'p': 54,
 'q': 55,
 'r': 56,
 's': 57,
 't': 58,
 'u': 59,
 'v': 60,
 'w': 61,
 'x': 62,
 'y': 63,
 'z': 64}

In [14]:
# all text is now represented as integer
text_as_int = np.array([char2idx[c] for c in t ])
text_as_int

array([18, 47, 56, ..., 45,  8,  0])

In [15]:
t[:13], text_as_int[:13]

('First Citizen', array([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52]))

In [16]:
t

"First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you know Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us kill him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be done: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citizens, the patricians good.\nWhat authority surfeits on would relieve us: if they\nwould yield us but the superfluity, while it were\nwholesome, we might guess they relieved us humanely;\nbut they think we are too dear: the leanness that\nafflicts us, the object of our misery, is as an\ninventory to particularise their abundance; our\nsufferance is a gain to them Let us revenge this with\nour pikes, ere we become rakes: for the gods know I\nspeak this in hunger 

In [17]:
len(t)

1115394

In [18]:
example_per_epoch= len(t)//(Config.seq_length+1)
example_per_epoch

11043

In [19]:
idx2char_dict ={val: key for key, val in char2idx.items()}
idx2char_dict

{0: '\n',
 1: ' ',
 2: '!',
 3: '$',
 4: '&',
 5: "'",
 6: ',',
 7: '-',
 8: '.',
 9: '3',
 10: ':',
 11: ';',
 12: '?',
 13: 'A',
 14: 'B',
 15: 'C',
 16: 'D',
 17: 'E',
 18: 'F',
 19: 'G',
 20: 'H',
 21: 'I',
 22: 'J',
 23: 'K',
 24: 'L',
 25: 'M',
 26: 'N',
 27: 'O',
 28: 'P',
 29: 'Q',
 30: 'R',
 31: 'S',
 32: 'T',
 33: 'U',
 34: 'V',
 35: 'W',
 36: 'X',
 37: 'Y',
 38: 'Z',
 39: 'a',
 40: 'b',
 41: 'c',
 42: 'd',
 43: 'e',
 44: 'f',
 45: 'g',
 46: 'h',
 47: 'i',
 48: 'j',
 49: 'k',
 50: 'l',
 51: 'm',
 52: 'n',
 53: 'o',
 54: 'p',
 55: 'q',
 56: 'r',
 57: 's',
 58: 't',
 59: 'u',
 60: 'v',
 61: 'w',
 62: 'x',
 63: 'y',
 64: 'z'}

In [20]:
idx2char= np.array(vocab)
idx2char

array(['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?',
       'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
       'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
       'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
       'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'],
      dtype='<U1')

In [21]:
# We can use any one of them

idx2char[0], idx2char_dict[0]

('\n', '\n')

In [22]:
print("Hi\nThere")

Hi
There


In [23]:
# getting exact representation of words

print(r"Hi\nThere")

Hi\nThere


In [24]:
# getting exact representation of words

print(repr("Hi\nThere"))

'Hi\nThere'


In [25]:
# Training example per target 
# every character is passed as a data 

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

# get tensors 
for i in char_dataset.take(10):
  print(i)

# get tensors 

tf.Tensor(18, shape=(), dtype=int64)
tf.Tensor(47, shape=(), dtype=int64)
tf.Tensor(56, shape=(), dtype=int64)
tf.Tensor(57, shape=(), dtype=int64)
tf.Tensor(58, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(15, shape=(), dtype=int64)
tf.Tensor(47, shape=(), dtype=int64)
tf.Tensor(58, shape=(), dtype=int64)
tf.Tensor(47, shape=(), dtype=int64)


In [26]:
for i in char_dataset.take(13):
  print(idx2char[i.numpy()])

# get tensors 

F
i
r
s
t
 
C
i
t
i
z
e
n


In [27]:
# Training example per target 
# creating batches
sequences = char_dataset.batch(Config.seq_length+1, drop_remainder= True)

for item in sequences.take(5):
  print(item.numpy())

[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43
 39 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49
  6  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10
  0 37 53 59  1]
[39 56 43  1 39 50 50  1 56 43 57 53 50 60 43 42  1 56 39 58 46 43 56  1
 58 53  1 42 47 43  1 58 46 39 52  1 58 53  1 44 39 51 47 57 46 12  0  0
 13 50 50 10  0 30 43 57 53 50 60 43 42  8  1 56 43 57 53 50 60 43 42  8
  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 18 47 56 57 58  6  1
 63 53 59  1 49]
[52 53 61  1 15 39 47 59 57  1 25 39 56 41 47 59 57  1 47 57  1 41 46 47
 43 44  1 43 52 43 51 63  1 58 53  1 58 46 43  1 54 43 53 54 50 43  8  0
  0 13 50 50 10  0 35 43  1 49 52 53 61  5 58  6  1 61 43  1 49 52 53 61
  5 58  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 24 43 58  1
 59 57  1 49 47]
[50 50  1 46 47 51  6  1 39 52 42  1 61 43  5 50 50  1 46 39 60 43  1 41


In [28]:
# Training example per target 
sequences = char_dataset.batch(Config.seq_length+1, drop_remainder= True)

for item in sequences.take(5):
  to_print= repr("".join([idx2char[c] for c in item.numpy()]))
  print(to_print)
  print("length of sentence: ", len(to_print))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
length of sentence:  110
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
length of sentence:  109
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
length of sentence:  109
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
length of sentence:  107
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'
length of sentence:  109


In [29]:
for item in sequences.take(5):
  print(repr("".join([idx2char[c] for c in item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [30]:
"".join([idx2char[c] for c in item.numpy()])

'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'

In [31]:
# split input and target
# First Citizen->irst Citizen

In [32]:
def split_input_target(chunk):
  input_text = chunk[:-1] # First Citizen
  target_text= chunk[1:]  # irst Citizen
  return input_text, target_text

dataset = sequences.map(split_input_target)

In [33]:
# trying to predict the next character
for input_example, target_example in dataset.take(1):
  print("input_data:\n")
  print(repr("".join([idx2char[i] for i in input_example.numpy()])))

  print("\ntarget_data:\n")
  print(repr("".join([idx2char[t] for t in target_example.numpy()])))

input_data:

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'

target_data:

'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [34]:
# drop_remainder means length can vary. If shorter length then drop the remaining
dataset= dataset.shuffle(Config.Buffer_size).batch(Config.Batch_size, drop_remainder=True)

dataset

<BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [35]:
Config.vocab_size= len(vocab)
Config.vocab_size

65

In [36]:
def build_model(
    vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
          tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape= [batch_size, None]),
          tf.keras.layers.GRU(rnn_units,
                              return_sequences= True,
                              stateful= True,
                              recurrent_initializer="glorot_uniform"),
          tf.keras.layers.Dense(vocab_size)
  ])

  return model

In [37]:
model= build_model(
    vocab_size= Config.vocab_size, 
    embedding_dim = Config.embedding_dim, 
    rnn_units= Config.rnn_units, 
    batch_size= Config.Batch_size
)

In [38]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           16640     
                                                                 
 gru (GRU)                   (64, None, 1024)          3938304   
                                                                 
 dense (Dense)               (64, None, 65)            66625     
                                                                 
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [39]:
# logits means raw values at the outcome
def loss(labels, logits): 
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits= True)

In [40]:
model.compile(optimizer="adam", loss=loss)

In [51]:
checkpoint_prefix= os.path.join(Config.checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath= checkpoint_prefix, 
    save_weights_only= True   )

In [42]:
history= model.fit(dataset, epochs= Config.EPOCHS, callbacks= [checkpoint_callback])

Epoch 1/30
172/172 [==============================] - 18s 61ms/step - loss: 2.6895
Epoch 2/30
172/172 [==============================] - 11s 58ms/step - loss: 1.9639
Epoch 3/30
172/172 [==============================] - 11s 59ms/step - loss: 1.6956
Epoch 4/30
172/172 [==============================] - 12s 60ms/step - loss: 1.5461
Epoch 5/30
172/172 [==============================] - 12s 61ms/step - loss: 1.4558
Epoch 6/30
172/172 [==============================] - 12s 62ms/step - loss: 1.3955
Epoch 7/30
172/172 [==============================] - 12s 64ms/step - loss: 1.3503
Epoch 8/30
172/172 [==============================] - 13s 66ms/step - loss: 1.3119
Epoch 9/30
172/172 [==============================] - 12s 63ms/step - loss: 1.2770
Epoch 10/30
172/172 [==============================] - 12s 65ms/step - loss: 1.2449
Epoch 11/30
172/172 [==============================] - 12s 64ms/step - loss: 1.2119
Epoch 12/30
172/172 [==============================] - 12s 63ms/step - loss: 1.1799
E

In [43]:
(len(text)/Config.Batch_size)/(Config.seq_length+1)

172.55476485148515

In [44]:
# Restoring checkpoint

tf.train.latest_checkpoint(Config.checkpoint_dir)

'./training_ckpt/ckpt_30'

In [45]:
Config.vocab_size= len(vocab)
Config.vocab_size

65

In [46]:
model_from_ckpt= build_model(
    vocab_size= Config.vocab_size, 
    embedding_dim = Config.embedding_dim, 
    rnn_units= Config.rnn_units, 
    batch_size= 1
)

In [47]:
model_from_ckpt.load_weights(tf.train.latest_checkpoint(Config.checkpoint_dir))

model_from_ckpt.build(tf.TensorShape([1, None]))


In [48]:
model_from_ckpt.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (1, None, 256)            16640     
                                                                 
 gru_1 (GRU)                 (1, None, 1024)           3938304   
                                                                 
 dense_1 (Dense)             (1, None, 65)             66625     
                                                                 
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


# Prediction

In [52]:
# define a 

def generate_text(model, start_string, no_of_chars_to_gen= 1000):
  # convert input text to no.
  input_val =[char2idx[s] for s in start_string]  # text converted into int
  input_val = tf.expand_dims(input_val, 0)         # []  ->> [1,]

  text_generated= list()

  temperature = 1.0

  model.reset_states()  # resetting the previous states if any while predictions.

  for i in range(no_of_chars_to_gen):
    predictions = model(input_val)
    
    predictions= tf.squeeze(predictions, 0)

    predictions= predictions/temperature
    print(predictions)
    predicted_id = tf.random.categorical(predictions, num_samples= 1)[-1,0].numpy() # using numpy will give int value
    print(predicted_id)

    input_val= tf.expand_dims([predicted_id], 0)
    text_generated.append(idx2char[predicted_id])

  return start_string +"".join(text_generated)

In [61]:
# , no_of_chars_to_gen= 10
result = generate_text(model=model_from_ckpt, start_string= "romeo")
print(result)

Streaming output truncated to the last 5000 lines.
   -5.0164757   -8.495841    -9.740205    -4.6287284  -11.3500805
   -5.8502593   -7.704401    -2.4627862   -3.7973897  -12.313899
   -5.877337   -11.959845    -3.3547933  -10.91442      2.150068
   -3.434308     1.2325023  -10.252476    12.158152    -7.2556314
   -4.169913     2.1796038    4.8533745   -4.0153513   -0.15046227
    3.118737    -0.53179747  -6.835099     5.129662    -3.2658184
   -7.1589804   -0.20453508  -2.3128994    6.484859     3.16318
   -5.0900025   -5.5704837   -7.696803     8.100634    -3.2843282 ]], shape=(1, 65), dtype=float32)
43
tf.Tensor(
[[ 11.146538    11.321771    10.002573    -8.867918    -6.9919615
    1.082249    11.178032     8.635216     8.803141    -4.2761245
    9.438782    10.335304    11.673651   -12.262089    -8.36719
   -7.853362    -6.090687    -8.158003    -7.8428836   -8.122767
   -6.4124928   -3.290462    -9.265617    -9.783807   -10.007599
   -7.880657    -9.588275    -4.8576303  -11.57767

In [54]:
tf.math.log([[0.5 ,0.5]])

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[-0.6931472, -0.6931472]], dtype=float32)>

In [55]:
samples = tf.random.categorical([[1.,2.,3.,4.]], 1)
samples.numpy()

array([[3]])

In [56]:
for _ in range(10):
  values = [[1., 2., 3., 4.]]
  samples = tf.random.categorical(values, 1)[-1, 0]

  print(samples.numpy())

2
3
3
2
1
3
1
3
3
3


In [57]:
for _ in range(10):
  predictons = [[1., 2., 3., 4.,55., 56., 100., 101., 200.]]
  samples = tf.random.categorical(values, 1)[-1, 0]

  print(samples.numpy())

3
3
1
3
3
2
3
3
3
3
